# Install packages
# labelme:LabelMe is an open-source graphical annotation tool for image and video data
# albumentation: albumentations is a Python library for image augmentation
# opencv:OpenCV is a Python library that allows you to perform image processing and computer vision tasks

In [ ]:
!pip install labelme tensorflow  opencv-python matplotlib albumentations

#  Collect Images Using OpenCV

In [ ]:
import os
import time
import uuid #A simple guide to generating random IDs using UUID in Python
import cv2 #It's open_cv's modules

In [ ]:
IMAGES_PATH = os.path.join('data','images') # here ; first we want to make "data" folder then we make "images" as file into "data" the we join this file and folder and save it in IMAGES_PATH
number_images = 30 #the number of images which webcam shot from our face in definite time duration

In [ ]:
cap = cv2.VideoCapture(0) ## video capturing with webcam / cv2.VideoCapture: turn your webcam on if you have more than one webcam you can index it which webcam you want to work
for imgnum in range(number_images): # difinite a for loop for take a shot from your reality webcam in exact time interval
    print('Collecting image {}'.format(imgnum))
    ret, frame = cap.read()   # cap.read : transfer the images into vector and vectorize it
                              #after taking a shot from face : ret is a boolean variable that returns true if the frame is available.
                              #frame is an image array vector captured based on the default frames per second
    imgname = os.path.join(IMAGES_PATH,f'{str(uuid.uuid1())}.jpg')  # at first here we put special id for each picture with uuid and change the id to string and format it with jpg and transfer  and join it to the IMAGES_PATH in images file
    cv2.imwrite(imgname, frame)  #  cv2.imwrite A function that is used for displaying a picture in a window
    cv2.imshow('frame', frame)
    time.sleep(0.5) # time.sleep : we use it to definite the time interval  of taking consecutively shot from face in different status 0.5 is the time that we have chosen here

    if cv2.waitKey(1) & 0xFF == ord('q'):#cv2. waitKey(1) & 0xFF==ord('q')This line means that when the user presses 'q' from the keyboard, our video will stop.
        break     #The waitKey(0) function returns -1 when no input is made whatsoever. As soon the event occurs i.e. a Button is pressed it returns a 32-bit integer.
                  #The 0xFF in this scenario is representing binary 11111111 a 8 bit binary, since we only require 8 bits to represent a character we AND waitKey(0) to 0xFF. As a result, an integer is obtained below 255.
                  #ord(char) returns the ASCII value of the character which would be again maximum 255.
                  #Hence by comparing the integer to the ord(char) value, we can check for a key pressed event and break the loop.###
cap.release( )#Once our work with the video is done, it is required that we release the resources that we have initialized for our code. For eg, if the VideoCapture object is using the webcam, then while it is using it, no other process on your system can use the webcam
cv2.destroyAllWindows() #close all windows at any time after exiting the script

#  Annotate Images with LabelMe you can refer to this link to know how working with labelme
# (https://datagen.tech/guides/image-annotation/labelme/)

In [ ]:
!labelme
 # now the labeme framework is openning and you can draw bounding box on images that you were shoting from your webcam first at data directory you have to make a label file next to the images file


# Review Dataset and Build Image Loading Function

In [ ]:
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt

###  Load Image into TF Data Pipeline
# tf.data: for transforming data to tensor shapes
# A training step involves the following steps:
# 1. File reading
# 2. Fetch or parse data
# 3. Data transformation
# 4. Using the data to train the model.



In [ ]:
images = tf.data.Dataset.list_files('data\\images\\*.jpg') # tf.data.Dataset.list_files: first put the all images(*) into list then transform it into tensors due to the top reasons which is mentioned

In [ ]:
images.as_numpy_iterator().next() # .as_numpy_iterator().next(): we can check that transforming is complete with this code because we in numpy and tensor form we have iteration on elements

In [ ]:
def load_image(x):
    byte_img = tf.io.read_file(x) # tf.io.read_file( filename, name=None ) Defined in generated file:Reads and outputs the entire contents of the input filename
    img = tf.io.decode_jpeg(byte_img) #Decode a JPEG-encoded image to a uint8 tensor.
    return img

In [ ]:
images = images.map(load_image) #map : implement the top function on  each element of tensor

In [ ]:
images.as_numpy_iterator().next()

In [ ]:
type(images)

#  View Raw Images with Matplotlib

In [ ]:
image_generator = images.batch(4).as_numpy_iterator() #take the images and put them in batches here we definite 4 batches

In [ ]:
plot_images = image_generator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, image in enumerate(plot_images): #enumerate: back index and images to idx and image
    ax[idx].imshow(image) ##imshow : display images
plt.show()

###  MANUALLY SPLT DATA INTO TRAIN TEST AND VAL

In [ ]:
90*.7 # 63 to train

In [ ]:
90*.15 # 14 and 13 to test and val

### Move the Matching Labels to the specific img files

In [ ]:
for folder in ['train','test','val']: # here we definite train test val folders in data
    for file in os.listdir(os.path.join('data', folder, 'images')):

        filename = file.split('.')[0]+'.json' #name of the images which we split it by . and save it in list and we need firt element of with  index zero beacuse second element is format of file that we add it '.json' manually
        existing_filepath = os.path.join('data','labels', filename)
        if os.path.exists(existing_filepath):
            new_filepath = os.path.join('data',folder,'labels',filename) #Move the Matching Labels to the specific img files
            os.replace(existing_filepath, new_filepath)

#  Apply Image Augmentation on Images and Labels using Albumentations

###  Setup Albumentations Transform Pipeline

In [ ]:
import albumentations as alb

In [ ]:
augmentor = alb.Compose([alb.RandomCrop(width=450, height=450),
                         alb.HorizontalFlip(p=0.5),
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2),
                         alb.RGBShift(p=0.2),
                         alb.VerticalFlip(p=0.5)],
                       bbox_params=alb.BboxParams(format='albumentations',
                                                  label_fields=['class_labels']))# alb.compose: augement our images to generate  more data for training  by changind the size/flip/brightness/contrast/gamma/rgb of images
                      #alb.BboxParams: we definite the bounding box which we drew with labelme


###  Load a Test Image and Annotation with OpenCV and JSON

In [ ]:
img = cv2.imread(os.path.join('data','train', 'images','ffd85fc5-cc1a-11ec-bfb8-a0cec8d2d278.jpg')) # first in data folder we make train test val file and in each file we make image and label files which labels are relating to the images
# and the join the directories with each other and with "imread" read the images is train file and one of the file which name is "ffd85fc5-cc1a-11ec-bfb8-a0cec8d2d278.jpg"

In [ ]:
with open(os.path.join('data', 'train', 'labels', 'ffd85fc5-cc1a-11ec-bfb8-a0cec8d2d278.json'), 'r') as f:
    label = json.load(f)  # here we work with labels of train images which format is json and we read json file with this code

In [ ]:
label['shapes'][0]['points'] # here is example that in json format we have shape and in subset of shape we need the first par with index zero which is related to coordinations of bounding box and the the points that we want which is our exact coordination

### Extract Coordinates and Rescale to Match Image Resolution

In [ ]:
coords = [0,0,0,0] # first we definite list with 4 elements because we have four thing for bounding box "x,y,w(width),h(height)"
coords[0] = label['shapes'][0]['points'][0][0] # now we change the each elements to related coordination here for example in label in shape part in json format and index zero related to coordination and point part we  have two by two matricies which each element represent each x,y,w,h
coords[1] = label['shapes'][0]['points'][0][1]
coords[2] = label['shapes'][0]['points'][1][0]
coords[3] = label['shapes'][0]['points'][1][1]

In [ ]:
coords

In [ ]:
coords = list(np.divide(coords, [640,480,640,480])) # now we normalize each coordination and by dividing and shorten the size of bbox and the save it into list

In [ ]:
coords

#  Apply Augmentations and View Results

In [ ]:
augmented = augmentor(image=img, bboxes=[coords], class_labels=['face']) # now we augment the pictures with class  'lable face' and bounding box list(coords) and img which is our images

In [ ]:
#augmented['bboxes'][0][:2] which augment our images which [0] is row one becuse in list(coords) we just have one row and [:2] it means the column one and coulumn two
#augmented['bboxes'][0][2:] which augment our images which [0] is row one becuse in list(coords) we just have one row and [2:] it means the column three and coulumn four

In [ ]:
augmented['bboxes']

In [ ]:
cv2.rectangle(augmented['image'], # cv2.rectangle: draw rectangle bounding box on each augmented pictures automaticaly according to 30 pictures that we manually took a shot from webcam
              tuple(np.multiply(augmented['bboxes'][0][:2], [450,450]).astype(int)),
              tuple(np.multiply(augmented['bboxes'][0][2:], [450,450]).astype(int)),
                    (255,0,0), 2)  ## notice that we should multipy by prior img scale to be showed by open_cv

plt.imshow(augmented['image'])

#  Build and Run Augmentation Pipeline

In [ ]:
for partition in ['train','test','val']:
    for image in os.listdir(os.path.join('data', partition, 'images')):
    # The os. listdir() method returns a list of the names of the entries in a directory
        img = cv2.imread(os.path.join('data', partition, 'images', image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join('data', partition, 'labels', f'{image.split(".")[0]}.json') # image.split(".")[0]: the label file and we split by . and save it into list which we need the index zero of that list
        if os.path.exists(label_path): # we check that te label of each  images in correct file according to for loop in partition that we are in  with  os.path.exists if its true we read label path and definite coordinations pipline for all of the images
            with open(label_path, 'r') as f:
                label = json.load(f)

            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [640,480,640,480]))

        try:
            for x in range(60): # now for each picture we  make 60 augmented pic from originial pic
                augmented = augmentor(image=img, bboxes=[coords], class_labels=['face'])
                cv2.imwrite(os.path.join('aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])#imwrite() function that saves an image object to a specified file here we save augmented['image'] in this file'aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'

                annotation = {} # now we definte annotation in dictionary form which includes such keys with names of bounding  box and (class of face: if the face is detected show 1 else show 0)
                annotation['image'] = image

                if os.path.exists(label_path): ## checking that related label exist in related file
                    if len(augmented['bboxes']) == 0:
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0
                    else:
                        annotation['bbox'] = augmented['bboxes'][0] #the coordination of bounding box
                        annotation['class'] = 1
                else:
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0


                with open(os.path.join('aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f) #dump function in Python is mainly used when we want to store and transfer objects (Python objects) into a file in the form of JSON
                                              # here we store annotation that we definite on the above in 'f' file that we open it as f

        except Exception as e: #'except Exception as e' statement is used with try-except statements to capture any exception raised in the try block and store it in an object named e
            print(e)

###  Load Augmented Images to Tensorflow Dataset (ETL)

In [ ]:
train_images = tf.data.Dataset.list_files('aug_data\\train\\images\\*.jpg', shuffle=False)## (Extraction: we transform of train images into tensor pipeline that we have mentioned on the above)
train_images = train_images.map(load_image)##(transforming to all images with map function)
train_images = train_images.map(lambda x: tf.image.resize(x, (120,120)))# resize the pixel of images
train_images = train_images.map(lambda x: x/255)## (load) #then normalization images by dividing by 255 and now we have pixel size between 0 and 1

In [ ]:
test_images = tf.data.Dataset.list_files('aug_data\\test\\images\\*.jpg', shuffle=False) # for test images
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (120,120)))
test_images = test_images.map(lambda x: x/255)

In [ ]:
val_images = tf.data.Dataset.list_files('aug_data\\val\\images\\*.jpg', shuffle=False) #for val images
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x, (120,120)))
val_images = val_images.map(lambda x: x/255)

In [ ]:
train_images.as_numpy_iterator().next()

#  Build Label Loading Function

In [ ]:
def load_labels(label_path): #Build Label Loading Function
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f: # we send the label path like:'aug_data\\train\\labels\\*.json' in function and open it( before we should change it to numpy format) and read it and encode it with utf-8
        label = json.load(f)

    return [label['class'], label['bbox']]

###  Load Labels to Tensorflow Dataset

tensor flow graph for tf.py_fubction to have less complexity of computation
(https://www.easy-tensorflow.com/files/1_2.png)



In [ ]:
train_labels = tf.data.Dataset.list_files('aug_data\\train\\labels\\*.json', shuffle=False) #train labels
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16])) # tf.py_function:this function allows expressing computations in a TensorFlow graph as Python functions. In particular, it wraps a Python function func in a once-differentiable TensorFlow operation that executes it with eager execution enabled
## mapping the unit8 type to labels and float16 to img to transform them into tf.data

In [ ]:
test_labels = tf.data.Dataset.list_files('aug_data\\test\\labels\\*.json', shuffle=False) # test labels
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
val_labels = tf.data.Dataset.list_files('aug_data\\val\\labels\\*.json', shuffle=False) #val labels
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
train_labels.as_numpy_iterator().next()

#  Combine Label and Image Samples

###  Create Final Datasets (Images/Labels)

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels)) #now we concat labels and images in each related file
train = train.shuffle(5000)
train = train.batch(8)
train = train.prefetch(4)##Prefetching is the technique of loading the next batch of data into memory before the current batch has finished processing 4 is the number of next batches

In [ ]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(1300)
test = test.batch(8)
test = test.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(1000)
val = val.batch(8)
val = val.prefetch(4)

In [ ]:
train.as_numpy_iterator().next()[1]

###  View Images and Annotations

In [ ]:
data_samples = train.as_numpy_iterator()

In [ ]:
res = data_samples.next()#res[0]:images list/res[1][0]:class label list/res[1][1]:coordinations of bounding box

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4):
    sample_image = res[0][idx]
    sample_coords = res[1][1][idx]

    cv2.rectangle(sample_image,
                  tuple(np.multiply(sample_coords[:2], [120,120]).astype(int)),
                  tuple(np.multiply(sample_coords[2:], [120,120]).astype(int)),
                        (255,0,0), 2) #cv2. rectangle(img,x,y,rgb index) method modifies the input image by drawing a rectangle on it according to the specified parameters

    ax[idx].imshow(sample_image)

###  Import Layers and Base Network

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

###  Download VGG16 (pretrained model for object detection)
vgg16 architecture in this link:(https://media.licdn.com/dms/image/C5612AQFcCggXwoU96A/article-inline_image-shrink_1000_1488/0/1589862621183?e=1697673600&v=beta&t=TLDAOctNTdUkmbdcW4r6AqK2gHyjajtph2nFktIbtC8)

In [ ]:
vgg = VGG16(include_top=False) ## include_top is the part of the pretrained model wich data is flatten and is redayto go to NN and classify it and soft max the output  he we don't need this

In [ ]:
vgg.summary()

###  Build instance of Network

In [ ]:
def build_model():
    input_layer = Input(shape=(120,120,3))

    vgg = VGG16(include_top=False)(input_layer)

    # Classification Model (class label 0 or 1)
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1)  ## input NN
    class2 = Dense(1, activation='sigmoid')(class1)  ##OUTPUT NN


    # Bounding box model (Regression(coordinations))
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2) ## input NN
    regress2 = Dense(4, activation='sigmoid')(regress1) ##OUTPUT NN

    facetracker = Model(inputs=input_layer, outputs=[class2, regress2])
    return facetracker

#  Test out Neural Network

In [ ]:
facetracker = build_model()

In [ ]:
facetracker.summary()

In [ ]:
X, y = train.as_numpy_iterator().next()

In [ ]:
X.shape

In [ ]:
classes, coords = facetracker.predict(X)

In [ ]:
classes, coords

#  Define Optimizer and LR

In [ ]:
batches_per_epoch = len(train)
lr_decay = (1./0.75 -1)/batches_per_epoch

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=lr_decay)

###  Create Localization Loss and Classification Loss
 Localization Loss equation:(https://https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT62F4efuNYPXCQjSY0aEX4XqR2zYJ3FFRFYQ&usqp=CAU)

In [ ]:
def localization_loss(y_true, yhat):  #y_true:the real value / yhat: the predicted valu
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2])) # tf.reduce_sum: summation / delta_coord(x,y)/ y_true: the coordination of bbox and y_true[:,:]is one row and and 4 columns(x,y,w,h) matrice

    h_true = y_true[:,3] - y_true[:,1]
    w_true = y_true[:,2] - y_true[:,0]

    h_pred = yhat[:,3] - yhat[:,1] ## prediction
    w_pred = yhat[:,2] - yhat[:,0] ## prediction

    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))  #delta_size(h,w)

    return delta_coord + delta_size

In [ ]:
classloss = tf.keras.losses.BinaryCrossentropy() ##BinaryCrossentropy The loss function which is functional in calassification issue
regressloss = localization_loss

# Test out Loss Metrics

In [ ]:
localization_loss(y[1], coords)  ##y[1] is coordination of bounding box

In [ ]:
classloss(y[0], classes)  ##y[0] is the binary calassification of detecting face which is 0 and 1

In [ ]:
regressloss(y[1], coords)

#  Train Neural Network

###  Create Custom Model Class

In [ ]:
class FaceTracker(Model):
    def __init__(self, eyetracker,  **kwargs):
        super().__init__(**kwargs)
        self.model = eyetracker

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt

    def train_step(self, batch, **kwargs):

        X, y = batch

        with tf.GradientTape() as tape:  ##GradientTape is a mathematical tool for automatic differentiation (autodiff)
            classes, coords = self.model(X, training=True)

            batch_classloss = self.closs(y[0], classes)
            batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords) #The "tf. cast" function casts a tensor to new type

            total_loss = batch_localizationloss+0.5*batch_classloss #we have constant term behind the eqution of localizationloss which we definite it as  0.5

            grad = tape.gradient(total_loss, self.model.trainable_variables)#When using gradient tape you pass model.trainable_weights which returns the weights and biases of the entire model and use the optimizer to apply the gradients.

        opt.apply_gradients(zip(grad, self.model.trainable_variables)) #update the gradient of trainable variables

        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}

    def test_step(self, batch, **kwargs):
        X, y = batch

        classes, coords = self.model(X, training=False)

        batch_classloss = self.closs(y[0], classes)
        batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
        total_loss = batch_localizationloss+0.5*batch_classloss

        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}

    def call(self, X, **kwargs):
        return self.model(X, **kwargs)

In [ ]:
model = FaceTracker(facetracker)

In [ ]:
model.compile(opt, classloss, regressloss)

###  Train
tensorboard shematic:(https://editor.analyticsvidhya.com/uploads/32892Capture.PNG)

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir) ##TensorBoard is a tool for providing the measurements and visualizations needed during the machine learning workflow

In [ ]:
hist = model.fit(train, epochs=10, validation_data=val, callbacks=[tensorboard_callback])

###  Plot Performance

In [ ]:
hist.history

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20,5))

ax[0].plot(hist.history['total_loss'], color='teal', label='loss')
ax[0].plot(hist.history['val_total_loss'], color='orange', label='val loss')
ax[0].title.set_text('Loss')
ax[0].legend()

ax[1].plot(hist.history['class_loss'], color='teal', label='class loss')
ax[1].plot(hist.history['val_class_loss'], color='orange', label='val class loss')
ax[1].title.set_text('Classification Loss')
ax[1].legend()

ax[2].plot(hist.history['regress_loss'], color='teal', label='regress loss')
ax[2].plot(hist.history['val_regress_loss'], color='orange', label='val regress loss')
ax[2].title.set_text('Regression Loss')
ax[2].legend()

plt.show()

###  Make Predictions on Test Set

In [ ]:
test_data = test.as_numpy_iterator()

In [ ]:
test_sample = test_data.next()

In [ ]:
yhat = facetracker.predict(test_sample[0])

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4):
    sample_image = test_sample[0][idx]
    sample_coords = yhat[1][idx]# yhat[0]:class label of face/ yhat[1]:coordinaton of test_image

    if yhat[0][idx] > 0.9: # if prediction of class label is bigger than 0.9  we can conclude that we have face in the image
        cv2.rectangle(sample_image,
                      tuple(np.multiply(sample_coords[:2], [120,120]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [120,120]).astype(int)),
                            (255,0,0), 2)

    ax[idx].imshow(sample_image)

###  Save the Model
The development of the model can be saved both before and after testing. As a result, a model will pick up where it left off to eliminate lengthy training periods. You can still share your model and have others replicate it if you save it. Most machine learning professionals share the following when publishing test models and techniques:

Code to create the model

The trained weights for the mode

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
facetracker.save('facetracker.h5') #h5 models include the updated weights and biases in  the model

In [ ]:
facetracker = load_model('facetracker.h5')

###  Real Time Detection
# **The real output in this link**:(https://user-images.githubusercontent.com/36365194/56557874-5b347e00-65ba-11e9-8b8f-48a9664b97e9.gif)

In [ ]:
cap = cv2.VideoCapture(0)# now model is completed and turn the webcam on and in real time model can detect faces
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:] #the boundary of image pixel size

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)#cv2. cvtColor() method is used to convert an image from one color space to another, BGR image is converted to RGB

    yhat = facetracker.predict(np.expand_dims(resized/255,0)) # numpy.expand_dims(a, axis)Expand the shape of an array.Insert a new axis that will appear at the axis position in the expanded array shape.
    sample_coords = yhat[1]

    if yhat[0] > 0.5: # check that class label bigger than 0.5 that show model can detect face by logistic regression algorithm beacuse here we have binary classification
        # Controls the main rectangle
        cv2.rectangle(frame,
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)),
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame,
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])),
                            (255,0,0), -1)

        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA) # cv2.putText(image, text, org, font, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])
                                                                              #Parameters:
                                                                              #image: It is the image on which text is to be drawn.
                                                                              #text: Text string to be drawn.
                                                                              #org: It is the coordinates of the bottom-left corner of the text string in the image. The coordinates are represented as tuples of two values i.e. (X coordinate value, Y coordinate value).
                                                                              #font: It denotes the font type. Some of font types are FONT_HERSHEY_SIMPLEX, FONT_HERSHEY_PLAIN, , etc.
                                                                              #fontScale: Font scale factor that is multiplied by the font-specific base size.
                                                                              #color: It is the color of text string to be drawn. For BGR, we pass a tuple. eg: (255, 0, 0) for blue color.
                                                                              #thickness: It is the thickness of the line in px.
                                                                              #lineType: This is an optional parameter.It gives the type of the line to be used.
                                                                              #bottomLeftOrigin: This is an optional parameter. When it is true, the image data origin is at the bottom-left corner. Otherwise, it is at the top-left corner.

    cv2.imshow('EyeTrack', frame) #cv2.imshow(window_name, image)
                                  #Parameters:
                                  #window_name: A string representing the name of the window in which image to be displayed.
                                  #image: It is the image that is to be displayed.

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()